<a href="https://colab.research.google.com/github/nagalakshmiavvuladodda/MODULE-1-LAB-2/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [10]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.33414932680538556


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [11]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [12]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [13]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33622079451886366
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


answers:
1.I believe there might be some confusion in your question. It seems like you are referring to a specific experiment or project involving data augmentation and accuracy improvement. However, I don't have access to specific experiments or data, and I don't have the ability to implement or test code.

If you are working on a machine learning project and trying to improve accuracy using data augmentation, here are some general guidelines:

Angle and Shear Constraints: The optimal values for angle and shear constraints can depend on your specific dataset and task. It's common to start with small values and gradually increase them to see how they impact model performance. There's no universal "best" value; it often requires experimentation.

Impact on Accuracy: The impact of augmentations on accuracy can vary. In some cases, augmentations can significantly improve model generalization by exposing it to a more diverse set of examples. However, the degree of improvement depends on factors such as the dataset size, the complexity of the task, and the quality of the augmentations.

Number of Augmentations: Increasing the number of augmentations from each sample can be beneficial to a certain extent. However, there's a trade-off, as too much augmentation may introduce noise or artifacts that could harm performance. It's recommended to experiment with different augmentation levels and monitor the validation performance.

Custom Augmentations: Implementing custom augmentations can be useful, especially if they are tailored to the characteristics of your dataset. Common augmentations include rotation, flipping, zooming, and changes in brightness or contrast. Experiment with combinations of these augmentations to find the most effective transformations for your specific task.

Combining Augmentations: Combining various augmentations in a pipeline can enhance the diversity of the training data. However, it's essential to strike a balance and avoid introducing unrealistic transformations that might confuse the model.

Highest Accuracy: The highest accuracy achievable depends on the complexity of your task, the quality of your data, and the architecture of your model. It's recommended to use techniques like cross-validation to get a more robust estimate of your model's performance.

Smallest Training Dataset: The minimum dataset size required for accuracy above 50% depends on the complexity of your task and the chosen model architecture. In general, more data usually leads to better performance, but data augmentation can help mitigate the effects of a small dataset.

To optimize your model's performance, it's crucial to iterate through different configurations, monitor performance metrics, and conduct thorough experimentation on your specific problem.

User
Does averaging the validation accuracy across multiple splits give more consistent results?
Does it give more accurate estimate of test accuracy?
What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
ChatGPT
Averaging the validation accuracy across multiple splits in a cross-validation setup can indeed provide more consistent and reliable results compared to relying on a single split. Here's why:

Consistency: Averaging over multiple splits helps reduce the impact of the randomness introduced by the initial split. Different splits may lead to variations in the data distribution between training and validation sets. Averaging over these splits provides a more stable estimate of model performance.

More Accurate Estimate: By using multiple splits, you get a better estimate of how well your model generalizes to different subsets of the data. This is especially important when the dataset is limited, and a single split might not be representative of the overall data distribution.

Effect of the Number of Iterations: The number of iterations or folds in cross-validation impacts the trade-off between computational cost and the reliability of the estimate. Typically, using a higher number of iterations (folds) provides a more robust estimate but comes at the cost of increased computation time. However, the improvement in estimate stability tends to diminish beyond a certain point.

Dealing with Small Datasets: While increasing the number of iterations in cross-validation can help in obtaining a more stable estimate of model performance, it doesn't directly address the limitations imposed by a very small training or validation dataset. Cross-validation helps you make the most of the available data by systematically evaluating the model across different subsets, but it doesn't create additional data.

In the context of dealing with a very small training dataset or validation dataset, it's often more beneficial to explore techniques such as data augmentation (as discussed in the previous response), transfer learning, or utilizing pre-trained models on larger datasets to leverage existing knowledge.

In summary, increasing the number of iterations in cross-validation improves the stability of the estimate, but it doesn't inherently solve the challenges associated with extremely small datasets. It's crucial to strike a balance between the number of iterations, computational resources, and the need for a reliable estimate of model performance.





